In [4]:
import time
from sklearn.datasets import load_digits
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
import catboost as cb
import plotly.express as px
from xgboost import XGBClassifier
from sklearn import metrics


if __name__ == '__main__':
    data = load_digits(as_frame=False)
    predicts = data.data
    target = data.target
    print('\nТаблица предикторов:\n')
    print(predicts)
    print('\nТаблица целевых переменных:\n')
    print(target)
    print('\nРазмерность таблицы предикторов:\n')
    print(predicts.shape)
    print('\nРазмерность таблицы целеых переменных:\n')
    print(target.shape)
    A_train, A_test, y_train, y_test = train_test_split(predicts, target, train_size=0.8)
    print('\nРазделение на тренировочную и тестовую выборки:\n')
    print(A_train.shape)
    print(A_test.shape)
    time_start = time.time()
    print('\nОбучение случайного леса без критерия останова:\n')
    random_forest = RandomForestClassifier(max_depth=15,min_samples_split=10).fit(A_train,y_train)

    y_preds_d = random_forest.predict(A_train)
    y_pred = random_forest.predict(A_test)
    foresttrain=f1_score(y_preds_d,y_train, average='macro')
    foresttest=f1_score(y_pred,y_test, average='macro')
    print("RandomForest\nTime used: ", time.time() - time_start)
    print('F1 мера для тренировочных данных {} % '.format(round((foresttrain * 100), 6)))
    print('F1 мера для тестовых данных {} % '.format(round((foresttest * 100), 6)))

    print('\nОбучение случайного используя критерия : перебор параметров и для каждой'
          'комбинации обучить модели, затем выбрать лучшую. Это можно сделать с помощью'
'GridSearchCV.\n')
    time_start = time.time()
    random_forest = RandomForestClassifier()
    params_grid = {"max_depth":[12,18], "min_samples_leaf":[1,10], "min_samples_split":[6,12]}
    grid_search_random_forest = GridSearchCV(estimator=random_forest, param_grid=params_grid, scoring="f1_macro",cv=4)

    grid_search_random_forest.fit(A_train,y_train)
    best_model = grid_search_random_forest.best_estimator_
    y_preds_d = best_model.predict(A_train)
    y_pred = best_model.predict(A_test)
    trainpercent=f1_score(y_preds_d,y_train, average='macro')
    testpercent=f1_score(y_pred,y_test, average='macro')
    print(best_model)
    print("RandomForest\nTime used: ", time.time() - time_start)
    print('F1 мера для тренировочных данных{} % '.format(round((trainpercent * 100), 6)))
    print('F1 мера для тестовых данных {} %'.format(round((testpercent * 100), 6)))
    fig = px.imshow(confusion_matrix(y_test, y_pred), text_auto=True)
    fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
    fig.show()
    print(metrics.confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    time_start = time.time()
    model_catboost_clf = cb.CatBoostClassifier(iterations=1000, task_type="GPU", devices='0')
    model_catboost_clf.fit(A_train, y_train)
    y_preds_t = model_catboost_clf.predict(A_train, task_type="CPU")
    y_preds = model_catboost_clf.predict(A_test, task_type="CPU")
    print("CatBoost(YANDEX)\nTime used: ", time.time() - time_start)
    #f1score=f1_score(y_preds_t,y_train, average='macro')
    #f1scoretest=f1_score(y_preds_t,y_test, average='macro')
    print('F1 мера для тренировочных данных ', f1_score(y_preds_t,y_train, average='macro'))
    print('F1 мера для тестовых данных ', f1_score(y_preds,y_test, average='macro'))
    fig = px.imshow(confusion_matrix(y_test, y_preds), text_auto=True)
    fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
    fig.show()
    print(metrics.confusion_matrix(y_test, y_preds))
    print(classification_report(y_test, y_preds))

    print('\n\nВыполнение третьего задания:\n')

    start = time.perf_counter()

    xgbCl = XGBClassifier()
    xgbCl.fit(A_train, y_train)
    test_predict = xgbCl.predict(A_test)
    xgbCl_score = xgbCl.score(A_test, y_test)
    time_start = time.time()
    print("Точность XGBoostClassifier модели равна: {} %".format(round((xgbCl_score * 100), 6)))
    print("\nМатрица ошибок XGBoostClassifier модели :")
    fig = px.imshow(confusion_matrix(y_test, test_predict), text_auto=True)
    fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
    fig.show()
    print(metrics.confusion_matrix(y_test, test_predict))

    y_predict = xgbCl.predict_proba(A_test)
    # print('Предсказанные данные:\n', y_predict)
    print("XgBoost\nTime used: ", time.time() - time_start)





Таблица предикторов:

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]

Таблица целевых переменных:

[0 1 2 ... 8 9 8]

Размерность таблицы предикторов:

(1797, 64)

Размерность таблицы целеых переменных:

(1797,)

Разделение на тренировочную и тестовую выборки:

(1437, 64)
(360, 64)

Обучение случайного леса без критерия останова:

RandomForest
Time used:  0.26980137825012207
F1 мера для тренировочных данных 99.930793 % 
F1 мера для тестовых данных 96.464986 % 

Обучение случайного используя критерия : перебор параметров и для каждойкомбинации обучить модели, затем выбрать лучшую. Это можно сделать с помощьюGridSearchCV.

RandomForestClassifier(max_depth=12, min_samples_split=6)
RandomForest
Time used:  13.375114440917969
F1 мера для тренировочных данных100.0 % 
F1 мера для тестовых данных 97.36529 %


[[30  0  0  0  1  0  0  0  0  0]
 [ 0 34  0  0  0  0  0  0  0  0]
 [ 0  0 37  0  0  0  0  0  0  0]
 [ 0  0  0 32  0  0  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 38  0  0  0  0]
 [ 1  0  0  0  0  0 51  0  0  0]
 [ 0  0  0  0  0  0  0 32  0  0]
 [ 0  2  1  0  0  0  0  0 30  0]
 [ 0  0  0  1  0  1  0  0  2 31]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        31
           1       0.94      1.00      0.97        34
           2       0.97      1.00      0.99        37
           3       0.97      1.00      0.98        32
           4       0.97      1.00      0.99        36
           5       0.97      1.00      0.99        38
           6       1.00      0.98      0.99        52
           7       1.00      1.00      1.00        32
           8       0.94      0.91      0.92        33
           9       1.00      0.89      0.94        35

    accuracy                           0.97       360
   macro avg       

[[31  0  0  0  0  0  0  0  0  0]
 [ 0 34  0  0  0  0  0  0  0  0]
 [ 0  0 37  0  0  0  0  0  0  0]
 [ 0  0  0 32  0  0  0  0  0  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  0  0  0  0 38  0  0  0  0]
 [ 1  1  0  0  0  0 50  0  0  0]
 [ 0  0  0  0  0  0  0 32  0  0]
 [ 0  2  1  0  0  1  0  0 29  0]
 [ 0  0  0  1  0  1  0  0  0 33]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        31
           1       0.92      1.00      0.96        34
           2       0.97      1.00      0.99        37
           3       0.97      1.00      0.98        32
           4       1.00      1.00      1.00        36
           5       0.95      1.00      0.97        38
           6       1.00      0.96      0.98        52
           7       1.00      1.00      1.00        32
           8       1.00      0.88      0.94        33
           9       1.00      0.94      0.97        35

    accuracy                           0.98       360
   macro avg       

[[31  0  0  0  0  0  0  0  0  0]
 [ 0 34  0  0  0  0  0  0  0  0]
 [ 0  0 37  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  1  0  0  1  0]
 [ 0  0  0  0 36  0  0  0  0  0]
 [ 0  1  0  0  0 37  0  0  0  0]
 [ 1  1  0  0  0  0 49  0  1  0]
 [ 0  0  0  0  0  0  0 32  0  0]
 [ 0  2  0  0  0  1  0  0 30  0]
 [ 0  0  0  0  0  1  0  0  1 33]]
XgBoost
Time used:  0.0780019760131836
